In [1]:
import os
import sys
import warnings
import numpy as np
import pandas as pd
import time
import datetime as dt
import re

from IPython.display import display, HTML, clear_output
from IPython.core.interactiveshell import InteractiveShell
import ipywidgets as widgets

from matplotlib import pyplot as plt
import seaborn as sns

from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings('ignore')
InteractiveShell.ast_node_interactivity = "all"
display(HTML("<style>.container { width:79% !important; }</style>"))

# easier-to-read notebook:
from IPython.display import display, HTML, clear_output
pd.options.display.max_columns = 70
pd.options.display.max_rows = 500
%load_ext autoreload
%autoreload 2

r = re.compile('.*maoz*.')
if len(list(filter(r.match, sys.path))) != 0:
    PHYTECH_DRIVE_PATH = os.environ['PHYTECH_DRIVE_PATH']
    GITHUB_PATH = PHYTECH_DRIVE_PATH + 'GitHub'
    DATA_WD = PHYTECH_DRIVE_PATH + '/Data Integrity/SM anomalies/data'
    CERT_PATH = PHYTECH_DRIVE_PATH + '/Data'
    if CERT_PATH not in sys.path:
        sys.path.append(CERT_PATH)
    from sql_import_export import SqlImporter
    import cert_aws as c
    sql_importer = SqlImporter(database = c.database_research, user = c.user_research, password = c.password_research,
                                host = c.host_research, port = c.port_research, verbose=True)

local aws_cert.py loaded


In [3]:
start_date = (dt.date.today() - dt.timedelta(days=8)).strftime("%Y-%m-%d")

query = f"""
    SELECT distinct(pm.project_id)
    FROM projects_metadata pm
    JOIN soil_sensors_metadata ssm
    ON pm.project_id = ssm.project_id
    JOIN project_irrigation_spans_v2 pis
    ON pis.project_id = pm.project_id
    WHERE time_zone like '%Los_Angeles'
    AND season = 2023
    AND type_id IN (90, 91, 92, 98, 117, 118, 124, 127, 135, 137)
    AND start_date >= CAST((CAST('{start_date}' AS timestamp)) AS date)
    AND active=true
    ORDER BY project_id
    """
sql_importer = SqlImporter(query=query, database=c.database_production, user=c.user_production, password=c.password_production,
                                host=c.host_production, port=c.port_production, verbose=True)

sql_importer.get_data()
project_list = sql_importer.data
project_list

Loaded table with 2293 lines from projects_metadata


,project_id
0,848545
1,848547
2,848548
3,849303
4,849304
...,...
2288,877954
2289,878012
2290,878013
2291,878566


In [6]:
query = f"""
            SELECT * --serial_number as sensor_id, updated_at, status
            FROM work_order_line_items
            --WHERE serial_number = 'self.sensor_id'
            --AND status not in ('closed','completed')
            order by updated_at desc
            limit 10
        """
sql_importer = SqlImporter(query=query, database=c.database_ruby_production, user=c.user_production, password=c.password_production, host=c.host_production, port=c.port_production)

sql_importer.get_data()
sql_importer.data
#sensor_support_status_dict = sql_importer.data.to_dict('records')

,id,product_type,product_name,sensor_type_id,status,work_type,serial_number,confirmed_serial_number,description,duration,work_order_id,sf_id,line_item_number,created_at,updated_at,deleted_at,depth,unit,manual,mandatory,quantity,product_id
0,1962195,sensor,SMP-6,98.0,None,install,None,None,None,None,547378,None,None,2023-05-09 07:37:01.631700,2023-05-09 07:37:01.631700,None,3.0,ft.,True,True,None,None
1,1961492,sensor,DER,74.0,closed,install,1097444,None,torn wire,None,458108,1WL6N000001D6iDWAS,None,2023-05-08 22:16:53.330998,2023-05-09 07:37:01.054705,None,,None,False,False,None,None
2,1961491,sensor,DER,74.0,closed,remove,1097444,None,torn wire,None,458108,1WL6N000001D6i8WAC,None,2023-05-08 22:16:53.211093,2023-05-09 07:37:01.044695,None,,None,False,False,None,None
3,1859881,sensor,SMP-6,98.0,closed,install,None,None,None,None,458108,1WL6N000001FDaKWAW,None,2023-03-19 07:10:22.724235,2023-05-09 07:37:01.039217,None,3.0,ft.,True,True,None,None
4,1962194,remote_logger,remote_logger,NaN,None,install,None,None,None,None,547377,None,None,2023-05-09 07:34:37.555275,2023-05-09 07:34:37.555275,None,None,None,False,None,None,None
5,1962190,sensor,SFI,141.0,new,install,None,None,None,None,547373,1WL6N000001D7KgWAK,None,2023-05-09 07:08:18.711811,2023-05-09 07:27:13.359763,None,None,None,True,True,None,None
6,1962189,sensor,SFI,141.0,new,install,None,None,None,None,547373,1WL6N000001D7KbWAK,None,2023-05-09 07:08:17.808381,2023-05-09 07:27:12.427822,None,None,None,True,True,None,None
7,1962188,sensor,SFI,141.0,new,install,None,None,None,None,547373,1WL6N000001D7KWWA0,None,2023-05-09 07:08:16.973633,2023-05-09 07:27:11.440316,None,None,None,True,True,None,None
8,1962187,remote_logger,remote_logger,NaN,new,battery,924217,None,None,None,547372,1WL6N000001D7KRWA0,None,2023-05-09 07:06:03.583887,2023-05-09 07:27:04.918871,None,None,None,True,True,None,None
9,1912361,sensor,TEN,71.0,completed,remove,1138937,None,battery,None,518874,1WL6N000001CQHuWAO,None,2023-04-12 12:09:57.513570,2023-05-09 07:17:13.653596,None,20.0,cm,False,False,None,None
